In [1]:
import json

In [43]:
with open("/content/codigo_civil.json", "r") as f:
    codigo = json.load(f)

def collect_artigo_and_header(data):
    result = {'articles': [], 'contexts': []}

    def traverse(data):
        if isinstance(data, dict):
            for key, value in data.items():
                if key == 'artigos' and value:
                    result['articles'].append(value)
                if key == 'header' and value:
                    result['contexts'].append(value)
                # Recursive call to handle nested dictionaries
                if isinstance(value, dict):
                    traverse(value)
                elif isinstance(value, list):
                    traverse(value)
        elif isinstance(data, list):
            for item in data:
                # Recursive call to handle lists of dictionaries
                traverse(item)

    traverse(data)
    return result

artigos = collect_artigo_and_header(codigo)['articles']

result = collect_artigo_and_header(codigo['parte especial']['LIVRO IV'])
iv_context = ''.join(result['contexts'])
iv_articles = result['articles']

result = collect_artigo_and_header(codigo['parte especial']['LIVRO V'])
v_context = ''.join(result['contexts'])
v_articles = result['articles']

iv_v_artigos = iv_articles + v_articles

In [27]:
def get_article(art_id, artigos=artigos):
    for art_group in artigos:
        for artigo in art_group:
            if art_id.replace('.', '') in artigo[:10].replace('.', ''): # limit to get article name in init
                return artigo.replace('\n\n', '').strip()

def convert_strlist_to_liststr(strlist):
    strlist = strlist.replace("'", "") # remove '
    liststr = strlist.split(", ") # split elements
    return liststr

In [5]:
import pandas as pd
df = pd.read_csv("/content/LLM1_QA_generation_clean.csv")
df.head()

,pergunta,resposta,justificativa,artigos,contexto,ref_pergunta,ref_resposta,ref_artigos
0,Quem não pode ser tutor?,Aqueles que não tiverem a livre administração ...,Essas pessoas não podem ser tutores porque não...,Art. 1.735,Art. 1.735. Não podem ser tutores e serão exon...,4. O cônjuge tem direito à pensão após o divór...,Apesar de menos com o pedido de pensão pelo cô...,['Art. 1.735. Não podem ser tutores e serão ex...
1,O que acontece quando o herdeiro aceita a hera...,A transmissão da herança torna-se definitiva.,A aceitação da herança implica a transmissão d...,Art. 1.804,"Art. 1.804. Aceita a herança, torna-se definit...",4. O cônjuge tem direito à pensão após o divór...,Apesar de menos com o pedido de pensão pelo cô...,['Art. 1.735. Não podem ser tutores e serão ex...
2,Como o herdeiro pode aceitar a herança?,A aceitação da herança pode ser expressa ou tá...,A aceitação da herança pode ser feita de forma...,Art. 1.805,"Art. 1.805. A aceitação da herança, quando exp...",4. O cônjuge tem direito à pensão após o divór...,Apesar de menos com o pedido de pensão pelo cô...,['Art. 1.735. Não podem ser tutores e serão ex...
3,O herdeiro pode renunciar a herança?,"Sim, o herdeiro pode renunciar a herança.","A renúncia da herança é possível, mas deve ser...",Art. 1.806,Art. 1.806. A renúncia da herança deve constar...,4. O cônjuge tem direito à pensão após o divór...,Apesar de menos com o pedido de pensão pelo cô...,['Art. 1.735. Não podem ser tutores e serão ex...
4,O que acontece se o herdeiro não se pronunciar...,Se o herdeiro não se pronunciar sobre a heranç...,Se o herdeiro não se pronunciar sobre a heranç...,Art. 1.807,Art. 1.807. O interessado em que o herdeiro de...,4. O cônjuge tem direito à pensão após o divór...,Apesar de menos com o pedido de pensão pelo cô...,['Art. 1.735. Não podem ser tutores e serão ex...


In [28]:
full_arts = []
for art_list in df['artigos']:
    row_arts = []
    for art in convert_strlist_to_liststr(art_list):
        full_art = get_article(art)
        row_arts.append(full_art)
    full_arts.append(row_arts)

len(full_arts)

290

In [29]:
df['artigos_completos'] = full_arts

In [31]:
df[['artigos_completos']]

,artigos_completos
0,[Art. 1.735. Não podem ser tutores e serão exo...
1,"[Art. 1.804. Aceita a herança, torna-se defini..."
2,"[Art. 1.805. A aceitação da herança, quando ex..."
3,[Art. 1.806. A renúncia da herança deve consta...
4,[Art. 1.807. O interessado em que o herdeiro d...
...,...
285,[Art. 1.524. As causas suspensivas da celebraç...
286,[Art. 1.871. O testamento pode ser escrito em ...
287,[Art. 1.798. Legitimam-se a suceder as pessoas...
288,[Art. 1.801. Não podem ser nomeados herdeiros ...


In [24]:
df

,pergunta,resposta,justificativa,artigos,contexto,ref_pergunta,ref_resposta,ref_artigos,artigos_completos
0,Quem não pode ser tutor?,Aqueles que não tiverem a livre administração ...,Essas pessoas não podem ser tutores porque não...,Art. 1.735,Art. 1.735. Não podem ser tutores e serão exon...,4. O cônjuge tem direito à pensão após o divór...,Apesar de menos com o pedido de pensão pelo cô...,['Art. 1.735. Não podem ser tutores e serão ex...,[Art. 173. O ato de confirmação deve conter a ...
1,O que acontece quando o herdeiro aceita a hera...,A transmissão da herança torna-se definitiva.,A aceitação da herança implica a transmissão d...,Art. 1.804,"Art. 1.804. Aceita a herança, torna-se definit...",4. O cônjuge tem direito à pensão após o divór...,Apesar de menos com o pedido de pensão pelo cô...,['Art. 1.735. Não podem ser tutores e serão ex...,"[Art. 180. O menor, entre dezesseis e dezoito ..."
2,Como o herdeiro pode aceitar a herança?,A aceitação da herança pode ser expressa ou tá...,A aceitação da herança pode ser feita de forma...,Art. 1.805,"Art. 1.805. A aceitação da herança, quando exp...",4. O cônjuge tem direito à pensão após o divór...,Apesar de menos com o pedido de pensão pelo cô...,['Art. 1.735. Não podem ser tutores e serão ex...,"[Art. 180. O menor, entre dezesseis e dezoito ..."
3,O herdeiro pode renunciar a herança?,"Sim, o herdeiro pode renunciar a herança.","A renúncia da herança é possível, mas deve ser...",Art. 1.806,Art. 1.806. A renúncia da herança deve constar...,4. O cônjuge tem direito à pensão após o divór...,Apesar de menos com o pedido de pensão pelo cô...,['Art. 1.735. Não podem ser tutores e serão ex...,"[Art. 180. O menor, entre dezesseis e dezoito ..."
4,O que acontece se o herdeiro não se pronunciar...,Se o herdeiro não se pronunciar sobre a heranç...,Se o herdeiro não se pronunciar sobre a heranç...,Art. 1.807,Art. 1.807. O interessado em que o herdeiro de...,4. O cônjuge tem direito à pensão após o divór...,Apesar de menos com o pedido de pensão pelo cô...,['Art. 1.735. Não podem ser tutores e serão ex...,"[Art. 180. O menor, entre dezesseis e dezoito ..."
...,...,...,...,...,...,...,...,...,...
285,Quem pode arguir as causas suspensivas da cele...,"Os parentes em linha reta de um dos nubentes, ...",O art. 1.524 do Código Civil Brasileiro estabe...,Art. 1.524,As causas suspensivas da celebração do casamen...,3) Quem fica com a guarda do menor em caso de ...,"Desde 2014, a Justiça prevê que os pais, se es...",['Art. 1.523. Não devem casar:\n\nI - o viúvo ...,"[Art. 152. No apreciar a coação, ter-se-ão em ..."
286,Quem pode fazer um testamento cerrado?,"Qualquer pessoa que saiba ler e escrever, exce...",Os arts. 1.871 e 1.873 do Código Civil Brasile...,"Art. 1.871, Art. 1.873",O testamento pode ser escrito em língua nacion...,3) Quem fica com a guarda do menor em caso de ...,"Desde 2014, a Justiça prevê que os pais, se es...",['Art. 1.523. Não devem casar:\n\nI - o viúvo ...,[Art. 1.871. O testamento pode ser escrito em ...
287,Quais são os legitimados a suceder?,As pessoas nascidas ou já concebidas no moment...,Os legitimados a suceder estão previstos nos A...,"Art. 1.798, Art. 1.799",Legitimam-se a suceder as pessoas nascidas ou ...,3) Quem fica com a guarda do menor em caso de ...,"Desde 2014, a Justiça prevê que os pais, se es...",['Art. 1.525. O requerimento de habilitação pa...,[Art. 1.798. Legitimam-se a suceder as pessoas...
288,Quem não pode ser nomeado herdeiro ou legatário?,"A pessoa que, a rogo, escreveu o testamento, n...",Os impedidos de serem nomeados herdeiros ou le...,Art. 1.801,Não podem ser nomeados herdeiros nem legatários:,3) Quem fica com a guarda do menor em caso de ...,"Desde 2014, a Justiça prevê que os pais, se es...",['Art. 1.525. O requerimento de habilitação pa...,"[Art. 180. O menor, entre dezesseis e dezoito ..."


In [36]:
acc = []
for i, art_str in enumerate(df['artigos']):
    arts = convert_strlist_to_liststr(art_str)
    for art in arts:
        if 'Art' in art:
            if ' e ' in art:
                art1, art2 = art.split(' e ')
                if art1 not in acc:
                    acc.append(art1.replace('Arts.', 'Art.'))
                if art2 not in acc:
                    if 'Art.' not in art2:
                        acc.append('Art. ' + art2)
            else:
                acc.append(art)

import natsort
len(natsort.natsorted(list(set(acc))))

217

In [44]:
sum([len(artigos_group) for artigos_group in iv_v_artigos])

518

In [40]:
1783-1511

272

In [41]:
2027-1784

243

In [42]:
273+244

517

In [45]:
217/517

0.4197292069632495

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


NameError: name 'pd' is not defined

In [21]:
import pandas as pd
df_eval = pd.read_csv('/content/LLM4_A_resp_score.csv')
df_eval

,justificativa,score,resposta_mais_completa,pergunta_original,resposta_original
0,A resposta original apresenta uma lista parcia...,1,De acordo com o Art. 1.735 do Código Civil Bra...,Quem não pode ser tutor?,Aqueles que não tiverem a livre administração ...
1,A resposta do FAQ apenas menciona que a transm...,1,"Quando o herdeiro aceita a herança, a transmis...",O que acontece quando o herdeiro aceita a hera...,A transmissão da herança torna-se definitiva.
2,"A resposta do FAQ é parcialmente correta, pois...",1,A aceitação da herança pode ser expressa ou tá...,Como o herdeiro pode aceitar a herança?,A aceitação da herança pode ser expressa ou tá...
3,A resposta é muito curta e não fornece informa...,1,"Sim, o herdeiro pode renunciar à herança, desd...",O herdeiro pode renunciar a herança?,"Sim, o herdeiro pode renunciar a herança."
4,"A resposta original não é completa, pois não m...",1,"De acordo com o Art. 1.807 do Código Civil, se...",O que acontece se o herdeiro não se pronunciar...,Se o herdeiro não se pronunciar sobre a heranç...
...,...,...,...,...,...
272,A resposta do FAQ apenas menciona uma das caus...,1,De acordo com o Art. 1.523 do Código Civil Bra...,Quem não pode casar?,O viúvo ou a viúva que tiver filho do cônjuge ...
273,"A resposta está correta, mas poderia ser mais ...",2,As causas suspensivas da celebração do casamen...,Quem pode arguir as causas suspensivas da cele...,"Os parentes em linha reta de um dos nubentes, ..."
274,A resposta original apresenta uma lista corret...,2,"De acordo com o Código Civil Brasileiro, os le...",Quais são os legitimados a suceder?,As pessoas nascidas ou já concebidas no moment...
275,"A resposta responde completamente à pergunta, ...",2,Não podem ser nomeados herdeiros nem legatário...,Quem não pode ser nomeado herdeiro ou legatário?,"A pessoa que, a rogo, escreveu o testamento, n..."


In [22]:
df_eval['score'].value_counts()

score
1    230
2     46
3      1
Name: count, dtype: int64

In [23]:
df_eval[df_eval['score']==1]

,justificativa,score,resposta_mais_completa,pergunta_original,resposta_original
0,A resposta original apresenta uma lista parcia...,1,De acordo com o Art. 1.735 do Código Civil Bra...,Quem não pode ser tutor?,Aqueles que não tiverem a livre administração ...
1,A resposta do FAQ apenas menciona que a transm...,1,"Quando o herdeiro aceita a herança, a transmis...",O que acontece quando o herdeiro aceita a hera...,A transmissão da herança torna-se definitiva.
2,"A resposta do FAQ é parcialmente correta, pois...",1,A aceitação da herança pode ser expressa ou tá...,Como o herdeiro pode aceitar a herança?,A aceitação da herança pode ser expressa ou tá...
3,A resposta é muito curta e não fornece informa...,1,"Sim, o herdeiro pode renunciar à herança, desd...",O herdeiro pode renunciar a herança?,"Sim, o herdeiro pode renunciar a herança."
4,"A resposta original não é completa, pois não m...",1,"De acordo com o Art. 1.807 do Código Civil, se...",O que acontece se o herdeiro não se pronunciar...,Se o herdeiro não se pronunciar sobre a heranç...
...,...,...,...,...,...
269,A resposta original não apresenta a completa d...,1,O casamento é a união entre dois cônjuges que ...,O que é o casamento?,O casamento é a união entre dois cônjuges que ...
270,A resposta original não é clara sobre quem pod...,1,"De acordo com o Art. 1.514 do Código Civil, o ...",Quem pode celebrar o casamento?,O casamento pode ser celebrado por qualquer pe...
271,"A resposta responde parcialmente à pergunta, p...",1,A exclusão do herdeiro ou legatário é feita po...,Como é feita a exclusão do herdeiro ou legatário?,A exclusão do herdeiro ou legatário é feita po...
272,A resposta do FAQ apenas menciona uma das caus...,1,De acordo com o Art. 1.523 do Código Civil Bra...,Quem não pode casar?,O viúvo ou a viúva que tiver filho do cônjuge ...


In [20]:
267/277

0.9638989169675091

In [24]:
import os

csvs = [file for file in os.listdir('.') if file.endswith('.csv')]
csvs

['LLM6_C_reab_score.csv',
 'LLM1_QA_generation_clean2.csv',
 'LLM7_AAC_coesion_score.csv',
 'LLM2_Q_score_v2.csv',
 'LLM4_A_resp_score.csv',
 'LLM8_Q_comp_score.csv',
 'LLM5_A_comp_score.csv',
 'LLM3_A_score_v2.csv']

In [25]:
csvs.remove('LLM1_QA_generation_clean2.csv')
csvs

['LLM6_C_reab_score.csv',
 'LLM7_AAC_coesion_score.csv',
 'LLM2_Q_score_v2.csv',
 'LLM4_A_resp_score.csv',
 'LLM8_Q_comp_score.csv',
 'LLM5_A_comp_score.csv',
 'LLM3_A_score_v2.csv']

In [26]:
csvs.sort()
csvs

['LLM2_Q_score_v2.csv',
 'LLM3_A_score_v2.csv',
 'LLM4_A_resp_score.csv',
 'LLM5_A_comp_score.csv',
 'LLM6_C_reab_score.csv',
 'LLM7_AAC_coesion_score.csv',
 'LLM8_Q_comp_score.csv']

In [27]:
df

,pergunta,resposta,justificativa,artigos,contexto,ref_pergunta,ref_resposta,ref_artigos,cluster
0,Quem não pode ser tutor?,Aqueles que não tiverem a livre administração ...,Essas pessoas não podem ser tutores porque não...,Art. 1.735,Art. 1.735. Não podem ser tutores e serão exon...,4. O cônjuge tem direito à pensão após o divór...,Apesar de menos com o pedido de pensão pelo cô...,['Art. 1.735. Não podem ser tutores e serão ex...,65
1,O que acontece quando o herdeiro aceita a hera...,A transmissão da herança torna-se definitiva.,A aceitação da herança implica a transmissão d...,Art. 1.804,"Art. 1.804. Aceita a herança, torna-se definit...",4. O cônjuge tem direito à pensão após o divór...,Apesar de menos com o pedido de pensão pelo cô...,['Art. 1.735. Não podem ser tutores e serão ex...,27
2,Como o herdeiro pode aceitar a herança?,A aceitação da herança pode ser expressa ou tá...,A aceitação da herança pode ser feita de forma...,Art. 1.805,"Art. 1.805. A aceitação da herança, quando exp...",4. O cônjuge tem direito à pensão após o divór...,Apesar de menos com o pedido de pensão pelo cô...,['Art. 1.735. Não podem ser tutores e serão ex...,21
3,O herdeiro pode renunciar a herança?,"Sim, o herdeiro pode renunciar a herança.","A renúncia da herança é possível, mas deve ser...",Art. 1.806,Art. 1.806. A renúncia da herança deve constar...,4. O cônjuge tem direito à pensão após o divór...,Apesar de menos com o pedido de pensão pelo cô...,['Art. 1.735. Não podem ser tutores e serão ex...,21
4,O que acontece se o herdeiro não se pronunciar...,Se o herdeiro não se pronunciar sobre a heranç...,Se o herdeiro não se pronunciar sobre a heranç...,Art. 1.807,Art. 1.807. O interessado em que o herdeiro de...,4. O cônjuge tem direito à pensão após o divór...,Apesar de menos com o pedido de pensão pelo cô...,['Art. 1.735. Não podem ser tutores e serão ex...,27
...,...,...,...,...,...,...,...,...,...
272,Quem não pode casar?,O viúvo ou a viúva que tiver filho do cônjuge ...,O art. 1.523 do Código Civil Brasileiro estabe...,Art. 1.523,Não devem casar: I - o viúvo ou a viúva que ti...,3) Quem fica com a guarda do menor em caso de ...,"Desde 2014, a Justiça prevê que os pais, se es...",['Art. 1.523. Não devem casar:\n\nI - o viúvo ...,65
273,Quem pode arguir as causas suspensivas da cele...,"Os parentes em linha reta de um dos nubentes, ...",O art. 1.524 do Código Civil Brasileiro estabe...,Art. 1.524,As causas suspensivas da celebração do casamen...,3) Quem fica com a guarda do menor em caso de ...,"Desde 2014, a Justiça prevê que os pais, se es...",['Art. 1.523. Não devem casar:\n\nI - o viúvo ...,6
274,Quais são os legitimados a suceder?,As pessoas nascidas ou já concebidas no moment...,Os legitimados a suceder estão previstos nos A...,"Art. 1.798, Art. 1.799",Legitimam-se a suceder as pessoas nascidas ou ...,3) Quem fica com a guarda do menor em caso de ...,"Desde 2014, a Justiça prevê que os pais, se es...",['Art. 1.525. O requerimento de habilitação pa...,15
275,Quem não pode ser nomeado herdeiro ou legatário?,"A pessoa que, a rogo, escreveu o testamento, n...",Os impedidos de serem nomeados herdeiros ou le...,Art. 1.801,Não podem ser nomeados herdeiros nem legatários:,3) Quem fica com a guarda do menor em caso de ...,"Desde 2014, a Justiça prevê que os pais, se es...",['Art. 1.525. O requerimento de habilitação pa...,65


In [28]:
criterios_score = [3, 2, 1, 2, 3, 4, 1] # usar >=
rst_df = pd.DataFrame()
for i, csv in enumerate(csvs):
    temp = pd.read_csv(csv)
    rst_df[csv] = temp['score'] >= criterios_score[i]
rst_df

,LLM2_Q_score_v2.csv,LLM3_A_score_v2.csv,LLM4_A_resp_score.csv,LLM5_A_comp_score.csv,LLM6_C_reab_score.csv,LLM7_AAC_coesion_score.csv,LLM8_Q_comp_score.csv
0,True,True,True,True,True,True,True
1,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True
3,True,True,True,True,True,True,True
4,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...
272,True,True,True,True,True,True,True
273,True,True,True,True,True,True,True
274,True,True,True,True,True,True,True
275,True,True,True,True,False,True,True


In [32]:
rst_df.sum(axis=1).value_counts()

7    245
6     26
5      3
4      3
Name: count, dtype: int64

In [45]:
rst_df['aprove'] = rst_df.sum(axis=1) >= 7
rst_df

,LLM2_Q_score_v2.csv,LLM3_A_score_v2.csv,LLM4_A_resp_score.csv,LLM5_A_comp_score.csv,LLM6_C_reab_score.csv,LLM7_AAC_coesion_score.csv,LLM8_Q_comp_score.csv,aprove
0,True,True,True,True,True,True,True,True
1,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True
3,True,True,True,True,True,True,True,True
4,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...
272,True,True,True,True,True,True,True,True
273,True,True,True,True,True,True,True,True
274,True,True,True,True,True,True,True,True
275,True,True,True,True,False,True,True,False


In [56]:
a = list(rst_df[rst_df['aprove']==False].index)
print(a)

[8, 23, 24, 42, 51, 60, 62, 63, 81, 82, 83, 103, 107, 113, 127, 131, 136, 143, 144, 162, 172, 176, 180, 217, 225, 237, 244, 251, 264, 270, 275, 276]


In [54]:
x = list(sample.index)
x.sort()
x

[30, 97, 126, 142, 144, 215, 220, 236, 255, 256]

In [57]:
[xe in a for xe in x]

[False, False, False, False, True, False, False, False, False, False]

In [33]:
245/277

0.8844765342960289

In [35]:
sample = df.sample(10, random_state=42)

In [36]:
x = [len(resposta) for resposta in sample['resposta']]

import numpy as np
np.mean(x)

115.2

In [37]:
sample

,pergunta,resposta,justificativa,artigos,contexto,ref_pergunta,ref_resposta,ref_artigos,cluster
30,O que acontece com o poder familiar em caso de...,A separação judicial ou divórcio não altera as...,A separação judicial ou divórcio não afeta o p...,Art. 1.632,"Art. 1.632: 'A separação judicial, o divórcio ...",5. Posso pedir danos morais caso o motivo do d...,Sim. O cônjuge pode ajuizar um pedido de danos...,['Art. 1.630. Os filhos estão sujeitos ao pode...,10
126,Como se faz a partilha dos bens da herança?,A partilha dos bens da herança pode ser feita ...,O art. 2.015 do Código Civil Brasileiro estabe...,Art. 2.015,"Se os herdeiros forem capazes, poderão fazer p...",2. Como fica os bens após o divórcio?,Em relação a partilha dos bens após o divórcio...,['Art. 1.548. É nulo o casamento contraído:\n\...,25
220,O que acontece se um herdeiro sonegar bens da ...,O herdeiro que sonegar bens da herança perderá...,A sonegação de bens da herança é uma conduta i...,Art. 1.992,"O herdeiro que sonegar bens da herança, não os...",3) Quem fica com a guarda do menor em caso de ...,"Desde 2014, a Justiça prevê que os pais, se es...","['Art. 1.634. Compete aos pais, quanto à pesso...",27
142,Quais são os requisitos para a celebração de u...,Os contraentes devem ser habilitados com a cer...,Os arts. 1.533 e 1.534 estabelecem os requisit...,Arts. 1.533 e 1.534,"Celebrar-se-á o casamento, no dia, hora e luga...",2. Posso deixar meus bens para quem eu quiser?,A lei brasileira estabelece que a herança pode...,"['Art. 1.533. Celebrar-se-á o casamento, no di...",56
255,Quais são as obrigações dos co-herdeiros após ...,Os co-herdeiros são reciprocamente obrigados a...,Os co-herdeiros devem indenizar-se mutuamente ...,Art. 2.024,Os co-herdeiros são reciprocamente obrigados a...,3) Quem fica com a guarda do menor em caso de ...,"Desde 2014, a Justiça prevê que os pais, se es...",['Art. 1.517. O homem e a mulher com dezesseis...,12
236,Como são atualizadas as prestações alimentícias?,"As prestações alimentícias, de qualquer nature...",A obrigação de prestar alimentos é recíproca e...,Art. 1.710,"As prestações alimentícias, de qualquer nature...",3) Quem fica com a guarda do menor em caso de ...,"Desde 2014, a Justiça prevê que os pais, se es...","['Art. 1.694. Podem os parentes, os cônjuges o...",54
97,Quais são as obrigações do curador quando o re...,O curador não é obrigado à prestação de contas...,O curador tem uma responsabilidade menor quand...,Art. 1.783,Quando o curador for o cônjuge e o regime de b...,2. Posso deixar meus bens para quem eu quiser?,A lei brasileira estabelece que a herança pode...,['Art. 1.781. As regras a respeito do exercíci...,11
215,É possível que um testamento seja válido mesmo...,"Sim, se o testador dispuser da sua metade, não...",O testador tem a liberdade de dispor da sua me...,Art. 1.975,"Não se rompe o testamento, se o testador dispu...",3) Quem fica com a guarda do menor em caso de ...,"Desde 2014, a Justiça prevê que os pais, se es...","['Art. 1.565. Pelo casamento, homem e mulher a...",24
256,Como é feita a indenização entre os co-herdeir...,A indenização é feita na proporção das quotas ...,O evicto será indenizado pelos co-herdeiros na...,Art. 2.026,O evicto será indenizado pelos co-herdeiros na...,3) Quem fica com a guarda do menor em caso de ...,"Desde 2014, a Justiça prevê que os pais, se es...",['Art. 1.517. O homem e a mulher com dezesseis...,89
144,Quais são os efeitos do casamento?,O casamento produz efeitos quanto ao estado do...,O art. 1.541 estabelece que o casamento produz...,Art. 1.541,O assento assim lavrado retrotrairá os efeitos...,2. Posso deixar meus bens para quem eu quiser?,A lei brasileira estabelece que a herança pode...,"['Art. 1.533. Celebrar-se-á o casamento, no di...",20


In [64]:
sample['prompt'] = 'Dado o contexto a seguir: ' + sample['contexto'] + '\nSobre o Código Civil Brasileiro responda a pergunta a seguir gerando uma resposta de aproximadamente 115 caracteres: ' + sample['pergunta']

In [65]:
sample[['prompt']]

,prompt
30,Dado o contexto a seguir: Art. 1.632: 'A separ...
126,Dado o contexto a seguir: Se os herdeiros fore...
220,Dado o contexto a seguir: O herdeiro que soneg...
142,Dado o contexto a seguir: Celebrar-se-á o casa...
255,Dado o contexto a seguir: Os co-herdeiros são ...
236,Dado o contexto a seguir: As prestações alimen...
97,Dado o contexto a seguir: Quando o curador for...
215,Dado o contexto a seguir: Não se rompe o testa...
256,Dado o contexto a seguir: O evicto será indeni...
144,Dado o contexto a seguir: Art. 1.642. Qualquer...


In [58]:
df.sample(1)

,pergunta,resposta,justificativa,artigos,contexto,ref_pergunta,ref_resposta,ref_artigos,cluster
85,Os cônjuges podem praticar atos de disposição ...,"Sim, os cônjuges podem praticar atos de dispos...","De acordo com o Art. 1.642 do Código Civil, os...",Art. 1.642,Art. 1.642. Qualquer que seja o regime de bens...,Há alguma exceção onde um enteado possa recebe...,"Sim, no caso de padrastros e madrastas que pod...","['Art. 1.639. É lícito aos nubentes, antes de ...",29


In [60]:
sample.loc[144, :] = df.loc[85, :]

In [61]:
sample.loc[144, 'prompt']

,pergunta,resposta,justificativa,artigos,contexto,ref_pergunta,ref_resposta,ref_artigos,cluster,prompt
30,O que acontece com o poder familiar em caso de...,A separação judicial ou divórcio não altera as...,A separação judicial ou divórcio não afeta o p...,Art. 1.632,"Art. 1.632: 'A separação judicial, o divórcio ...",5. Posso pedir danos morais caso o motivo do d...,Sim. O cônjuge pode ajuizar um pedido de danos...,['Art. 1.630. Os filhos estão sujeitos ao pode...,10,Sobre o Código Civil Brasileiro responda a per...
126,Como se faz a partilha dos bens da herança?,A partilha dos bens da herança pode ser feita ...,O art. 2.015 do Código Civil Brasileiro estabe...,Art. 2.015,"Se os herdeiros forem capazes, poderão fazer p...",2. Como fica os bens após o divórcio?,Em relação a partilha dos bens após o divórcio...,['Art. 1.548. É nulo o casamento contraído:\n\...,25,Sobre o Código Civil Brasileiro responda a per...
220,O que acontece se um herdeiro sonegar bens da ...,O herdeiro que sonegar bens da herança perderá...,A sonegação de bens da herança é uma conduta i...,Art. 1.992,"O herdeiro que sonegar bens da herança, não os...",3) Quem fica com a guarda do menor em caso de ...,"Desde 2014, a Justiça prevê que os pais, se es...","['Art. 1.634. Compete aos pais, quanto à pesso...",27,Sobre o Código Civil Brasileiro responda a per...
142,Quais são os requisitos para a celebração de u...,Os contraentes devem ser habilitados com a cer...,Os arts. 1.533 e 1.534 estabelecem os requisit...,Arts. 1.533 e 1.534,"Celebrar-se-á o casamento, no dia, hora e luga...",2. Posso deixar meus bens para quem eu quiser?,A lei brasileira estabelece que a herança pode...,"['Art. 1.533. Celebrar-se-á o casamento, no di...",56,Sobre o Código Civil Brasileiro responda a per...
255,Quais são as obrigações dos co-herdeiros após ...,Os co-herdeiros são reciprocamente obrigados a...,Os co-herdeiros devem indenizar-se mutuamente ...,Art. 2.024,Os co-herdeiros são reciprocamente obrigados a...,3) Quem fica com a guarda do menor em caso de ...,"Desde 2014, a Justiça prevê que os pais, se es...",['Art. 1.517. O homem e a mulher com dezesseis...,12,Sobre o Código Civil Brasileiro responda a per...
236,Como são atualizadas as prestações alimentícias?,"As prestações alimentícias, de qualquer nature...",A obrigação de prestar alimentos é recíproca e...,Art. 1.710,"As prestações alimentícias, de qualquer nature...",3) Quem fica com a guarda do menor em caso de ...,"Desde 2014, a Justiça prevê que os pais, se es...","['Art. 1.694. Podem os parentes, os cônjuges o...",54,Sobre o Código Civil Brasileiro responda a per...
97,Quais são as obrigações do curador quando o re...,O curador não é obrigado à prestação de contas...,O curador tem uma responsabilidade menor quand...,Art. 1.783,Quando o curador for o cônjuge e o regime de b...,2. Posso deixar meus bens para quem eu quiser?,A lei brasileira estabelece que a herança pode...,['Art. 1.781. As regras a respeito do exercíci...,11,Sobre o Código Civil Brasileiro responda a per...
215,É possível que um testamento seja válido mesmo...,"Sim, se o testador dispuser da sua metade, não...",O testador tem a liberdade de dispor da sua me...,Art. 1.975,"Não se rompe o testamento, se o testador dispu...",3) Quem fica com a guarda do menor em caso de ...,"Desde 2014, a Justiça prevê que os pais, se es...","['Art. 1.565. Pelo casamento, homem e mulher a...",24,Sobre o Código Civil Brasileiro responda a per...
256,Como é feita a indenização entre os co-herdeir...,A indenização é feita na proporção das quotas ...,O evicto será indenizado pelos co-herdeiros na...,Art. 2.026,O evicto será indenizado pelos co-herdeiros na...,3) Quem fica com a guarda do menor em caso de ...,"Desde 2014, a Justiça prevê que os pais, se es...",['Art. 1.517. O homem e a mulher com dezesseis...,89,Sobre o Código Civil Brasileiro responda a per...
144,Os cônjuges podem praticar atos de disposição ...,"Sim, os cônjuges podem praticar atos de dispos...","De acordo co

In [66]:
gpt_ans = """
Em caso de separação judicial ou divórcio, o poder familiar continua sendo exercido por ambos os pais.

A partilha dos bens da herança é feita de acordo com a lei ou o testamento, distribuindo os bens entre os herdeiros.

Se um herdeiro sonegar bens da herança, ele perde o direito sobre esses bens e pode ser penalizado legalmente.

Os requisitos para a celebração do casamento incluem capacidade civil, consentimento mútuo e observância das formalidades legais.

Após a partilha, os co-herdeiros são responsáveis pelos encargos da herança e devem cumprir as disposições estabelecidas.

As prestações alimentícias são atualizadas conforme índices oficiais de correção monetária ou conforme estipulado em sentença.

No regime de comunhão universal, o curador deve administrar os bens do curatelado e prestar contas ao juiz periodicamente.

Sim, um testamento pode ser válido sem contemplar herdeiros necessários, desde que respeite a legítima, que é a parte da herança reservada a eles.

A indenização entre co-herdeiros em caso de evicção é proporcional às partes de cada um na herança, conforme o Código Civil.

Sim, os cônjuges podem praticar atos de disposição e administração necessários ao desempenho de suas profissões, conforme o Código Civil Brasileiro.
""".strip().split('\n\n')

In [67]:
gpt_ans

['Em caso de separação judicial ou divórcio, o poder familiar continua sendo exercido por ambos os pais.',
 'A partilha dos bens da herança é feita de acordo com a lei ou o testamento, distribuindo os bens entre os herdeiros.',
 'Se um herdeiro sonegar bens da herança, ele perde o direito sobre esses bens e pode ser penalizado legalmente.',
 'Os requisitos para a celebração do casamento incluem capacidade civil, consentimento mútuo e observância das formalidades legais.',
 'Após a partilha, os co-herdeiros são responsáveis pelos encargos da herança e devem cumprir as disposições estabelecidas.',
 'As prestações alimentícias são atualizadas conforme índices oficiais de correção monetária ou conforme estipulado em sentença.',
 'No regime de comunhão universal, o curador deve administrar os bens do curatelado e prestar contas ao juiz periodicamente.',
 'Sim, um testamento pode ser válido sem contemplar herdeiros necessários, desde que respeite a legítima, que é a parte da herança reservad

In [69]:
sample['resposta'].tolist()

['A separação judicial ou divórcio não altera as relações entre pais e filhos, exceto quanto ao direito de terem os filhos em sua companhia.',
 'A partilha dos bens da herança pode ser feita amigavelmente, por escritura pública, termo nos autos do inventário, ou escrito particular, homologado pelo juiz.',
 'O herdeiro que sonegar bens da herança perderá o direito que sobre eles lhe cabia.',
 'Os contraentes devem ser habilitados com a certidão do art. 1.531 e a solenidade deve ser realizada na sede do cartório, com toda publicidade, a portas abertas, presentes pelo menos duas testemunhas.',
 'Os co-herdeiros são reciprocamente obrigados a indenizar-se no caso de evicção dos bens aquinhoados.',
 'As prestações alimentícias, de qualquer natureza, serão atualizadas segundo índice oficial regularmente estabelecido.',
 'O curador não é obrigado à prestação de contas, salvo determinação judicial.',
 'Sim, se o testador dispuser da sua metade, não contemplando os herdeiros necessários de cuja

In [74]:
from nltk.translate.bleu_score import sentence_bleu

references = [[ans.split()] for ans in gpt_ans]
candidates = [ans.split() for ans in sample['resposta'].tolist()]

# Calcular BLEU Score para cada par de referência-candidato
for ref_list, candidate in zip(references, candidates):
    bleu_score = sentence_bleu(ref_list, candidate)
    print(f'BLEU Score para candidato {candidate}: {bleu_score:.4f}')

BLEU Score para candidato ['A', 'separação', 'judicial', 'ou', 'divórcio', 'não', 'altera', 'as', 'relações', 'entre', 'pais', 'e', 'filhos,', 'exceto', 'quanto', 'ao', 'direito', 'de', 'terem', 'os', 'filhos', 'em', 'sua', 'companhia.']: 0.0000
BLEU Score para candidato ['A', 'partilha', 'dos', 'bens', 'da', 'herança', 'pode', 'ser', 'feita', 'amigavelmente,', 'por', 'escritura', 'pública,', 'termo', 'nos', 'autos', 'do', 'inventário,', 'ou', 'escrito', 'particular,', 'homologado', 'pelo', 'juiz.']: 0.2083
BLEU Score para candidato ['O', 'herdeiro', 'que', 'sonegar', 'bens', 'da', 'herança', 'perderá', 'o', 'direito', 'que', 'sobre', 'eles', 'lhe', 'cabia.']: 0.0000
BLEU Score para candidato ['Os', 'contraentes', 'devem', 'ser', 'habilitados', 'com', 'a', 'certidão', 'do', 'art.', '1.531', 'e', 'a', 'solenidade', 'deve', 'ser', 'realizada', 'na', 'sede', 'do', 'cartório,', 'com', 'toda', 'publicidade,', 'a', 'portas', 'abertas,', 'presentes', 'pelo', 'menos', 'duas', 'testemunhas.']: 